### Importing dependencies

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import google.drive as drive
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision.transforms import Normalize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
from torchvision.transforms import Resize
from torchvision.transforms import GaussianBlur
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.metrics import confusion_matrix

## Loading and Pre Processing Data

Resizing the images (so they each have the same amount of pixels, not to overfit to larger images) and converting them to Tensor, so they can be used in the model.

Import the data from Google Drive as we assume that this is running on Google Colab.

In [ ]:
drive.mount('/content/drive')

In [ ]:
means = [0.4375447928905487, 0.4979252815246582, 0.3755785822868347]
stds = [0.17900893092155457, 0.16678600013256073, 0.17380985617637634]
transformTrain = Compose([Resize(size=(150,150)), GaussianBlur(), ToTensor(),Normalize(mean=means, std=stds)])
transformValTest = Compose([Resize(size=(150,150)), GaussianBlur(), ToTensor(),Normalize(mean=means, std=stds)])

train_dataset = ImageFolder(root="/content/drive/MyDrive/splitted_data/train",transform=transformTrain)
val_dataset = ImageFolder(root="/content/drive/MyDrive/splitted_data/val",transform=transformValTest)
test_dataset = ImageFolder(root="/content/drive/MyDrive/splitted_data/test",transform=transformValTest)

### Use a GPU if available

In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

device = get_default_device()
device

### Defining functions to evaluate the model performance

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

## Defining the ResNet9 Model Architecture

In [ ]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
              nn.BatchNorm2d(out_channels),
              nn.ReLU(inplace = True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()

        self.conv1 = conv_block(in_channels, 64)                               # 64  x 150 x 150
        self.conv2 = conv_block(64, 128, pool=True)                            # 128 x 75  x 75
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))  # 128 x 75  x 75

        self.conv3 = conv_block(128, 256, pool=True)                           # 256 x 37  x 37
        self.conv4 = conv_block(256, 512, pool=True)                           # 512 x 18  x 18
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))  # 512 x 18  x 18

        self.classifier = nn.Sequential(nn.MaxPool2d(18),                      # 512 x 1   x 1
                                        nn.Flatten(),                          # 512
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes))           # 512 --> 10

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

### Move data and the model to the GPU

In [ ]:
train_dl=DeviceDataLoader(DataLoader(train_dataset,pin_memory=True, batch_size=16),device)
val_dl=DeviceDataLoader(DataLoader(val_dataset,pin_memory=True, batch_size=16),device)
test_dl=DeviceDataLoader(DataLoader(test_dataset,pin_memory=True, batch_size=16),device)

model=to_device(ResNet9(3,6),device)
model

### Preparing the training loop

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader,
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []


    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs,steps_per_epoch=len(train_loader))

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            lrs.append(get_lr(optimizer))
            sched.step()

        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

history=[evaluate(model,val_dl)]
history

### Train the model using One Cycle Learning Rate Policy

Parameters:

In [ ]:
epochs=10
max_lr=0.01
grad_clip=0.1
weight_decay=1e-4
opt_func= torch.optim.Adam

In [ ]:
%time

history+=fit_one_cycle(epochs,max_lr,model,train_dl,val_dl,grad_clip=grad_clip,weight_decay=weight_decay,opt_func=opt_func)

### Show the plots of the training and validation losses

In [ ]:
def plot_accuracies(history):
  accuracies = [x.get('val_acc') for x in history]
  plt.plot(accuracies,'-x')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.title("Accuracy Vs No. of Epochs")

def plot_losses(history):
  train_losses = [x.get('train_loss') for x in history]
  val_losses = [x['val_loss'] for x in history]
  plt.plot(train_losses,'-bx')
  plt.plot(val_losses,'-rx')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend(['Training','Validation'])
  plt.title("Loss Vs No. of Epochs")

def plot_lrs(history):
  lrs = np.concatenate([x.get('lrs',[]) for x in history])
  plt.plot(lrs)
  plt.xlabel('Batch no.')
  plt.ylabel('Learning Rate')
  plt.title('Learning Rate Vs Batch No.')

print(plot_accuracies(history))
print(plot_losses(history))
print(plot_lrs(history))

### Evaluate the model with the test set

In [ ]:
def get_predictions(model, data_loader):
    model.eval()
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for batch in data_loader:
            images, targets = batch
            predictions = model(images)
            _, predicted_labels = torch.max(predictions, dim=1)
            all_predictions.extend(predicted_labels.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    return all_predictions, all_targets

def calculate_metrics(predictions, targets):
    cm = confusion_matrix(targets, predictions)
    recall = np.diag(cm) / np.sum(cm, axis=1)
    precision = np.diag(cm) / np.sum(cm, axis=0)
    return cm, recall, precision

predictions, targets = get_predictions(model, test_dl)
cm, recall, precision = calculate_metrics(predictions, targets)
print("Confusion Matrix:")
print(cm)
print("Recall:", recall)
print("Precision:", precision)